<a href="https://colab.research.google.com/github/Samantha996/Machine-Learning/blob/main/StockPredictionApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This webapp using StreamLit and Facebook Prophet

In [ ]:
!pip install streamlit
!pip install fbprophet
!pip install yfinance
!pip install pyngrok==4.1.1 #needed to create streamlit app through colaboratory
!pip install plotly 

     |████████████████████████████████| 7.5MB 5.7MB/s 
     |████████████████████████████████| 163kB 42.1MB/s 
     |████████████████████████████████| 112kB 44.5MB/s 
     |████████████████████████████████| 4.6MB 40.1MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 122kB 45.9MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=5f816914270fb6d2e8c2fbdafdc9d1a8729a08691748ff01aa8f9f82a327a658
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


     |████████████████████████████████| 5.5MB 6.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=8872c45dfd4077f8450b9b7a99d580ccc14b33a1b21702d7b2c635b999a3668a
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp36-none-any.whl size=15971 sha256=e16906044bed13ee49fae1db1cbe0b8b9aca8caef20017a6a116ba5e70ae8472
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [ ]:
%%writefile app.py

#importing libraries
import yfinance as yf
import datetime
import streamlit as st
from datetime import date
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs as go # interactive graphs

#establishing dates (2015 to present day)
date = datetime.date.today()
START = "2015-01-01" # stock start date
TODAY = date.today().strftime("%Y-%m-%d")

#choosing stocks
st.title("Stock Prediction App")
stocks = ("TSLA", "AAPL", "GOOG", "MSFT", "GME", "VRM", "WFC", "SNOW", "NFLX", "CCIV", "ROKU", "TWLO")
selected_stocks = st.selectbox("Select dataset for prediction", stocks)

n_years = st.slider("Years of Prediction:", 1 , 4)
period = n_years * 365 

#function which goes through the different tickers and saves the data
@st.cache #cache data 
def load_data(ticker):
  data = yf.download(ticker, START, TODAY)
  data.reset_index(inplace=True) # date in first column
  return data

data_load_state = st.text("Loading data...")
data = load_data(selected_stocks)
data_load_state.text("Loading data...done!")

#displays data
st.subheader('Raw Data')
st.write(data.tail()) #raw data 

#plot current data
def plot_raw_data():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name = 'stock_open'))
  fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name='stock_close'))
  fig.layout.update(title_text="Time Series Data", xaxis_rangeslider_visible=True)
  st.plotly_chart(fig)

plot_raw_data()

#Forecasting with FB Prophet
df_train = data[['Date', 'Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

st.subheader('Forecast data')
st.write(forecast.tail())

st.write(f'Forecast Plot for {n_years} years')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

st.write('Forecast Components')
fig2 = m.plot_components(forecast)
st.write(fig2)

Writing app.py


In [ ]:
#needed to create local host
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

http://21788c4b891f.ngrok.io
2021-02-22 01:44:48.400 NumExpr defaulting to 2 threads.
2021-02-22 01:44:50.537 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2021-02-22 01:47:19.795 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2021-02-22 01:47:25.742 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2021-02-22 01:53:45.972 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2021-02-22 01:54:41.541 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Credit for App - Python Engineer
https://www.youtube.com/watch?v=0E_31WqVzCY 